In [1]:
import sys

sys.path.append('../GraphStructureLearning')

In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import numpy as np
import pickle

from torch_geometric.data import Data as gData
from torch_geometric.utils import to_networkx, to_undirected
from torch_geometric.nn import MessagePassing

import networkx as nx
import matplotlib.pyplot as plt

from torch_geometric_temporal.nn.recurrent import DCRNN

# GTS

In [3]:
from models.GTS.gts_graph_learning import GTS_Graph_Learning
from models.GTS.gts_forecasting_module import GTS_Forecasting_Module
from models.GTS.DCRNN import DCRNN
from utils.utils import build_fully_connected_edge_idx

from glob import glob
import yaml
from easydict import EasyDict as edict

In [4]:
config_file = glob('./config/GTS/*.yaml')[0]
config = edict(yaml.load(open(config_file, 'r'), Loader=yaml.FullLoader))

In [5]:
# node_feas = torch.rand(config.nodes_num, config.node_features, 1000)
# edge_index = build_fully_connected_edge_idx(num_nodes=config.nodes_num)

In [6]:
total = np.load('./data/gts_featuremat.npy')

In [7]:
total.shape

(100, 48000)

In [8]:
spike = pickle.load(open('./data/spk_bin_n100.pickle', 'rb'))

spike = torch.FloatTensor(spike[:,:48000])

In [9]:
config.nodes_num = 100

In [10]:
config.graph_learning.stride = [20, 10, 5]
config.graph_learning.kernel_size = [200, 50, 10]

In [11]:
edge_index = build_fully_connected_edge_idx(num_nodes=config.nodes_num)

In [12]:
edge_index.shape

torch.Size([2, 9900])

In [13]:
spike.shape

torch.Size([100, 48000])

In [14]:
gl = GTS_Graph_Learning(config)

In [15]:
gl

GTS_Graph_Learning(
  (conv1): Conv1d(1, 8, kernel_size=(200,), stride=(20,))
  (conv2): Conv1d(8, 16, kernel_size=(50,), stride=(10,))
  (conv3): Conv1d(16, 64, kernel_size=(10,), stride=(5,))
  (fc_conv): Conv1d(64, 1, kernel_size=(1,), stride=(1,))
  (fc): Linear(in_features=46, out_features=64, bias=True)
  (hidden_drop): Dropout(p=0.2, inplace=False)
  (bn1): BatchNorm1d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn3): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc_cat): Linear(in_features=128, out_features=64, bias=True)
  (fc_out): Linear(in_features=64, out_features=2, bias=True)
)

In [69]:
adj = gl(spike, edge_index)

torch.Size([100, 8, 2391])
torch.Size([100, 16, 235])
torch.Size([100, 64, 46])
torch.Size([100, 1, 46])


In [70]:
adj.shape

torch.Size([9900, 2])

In [71]:
z_1 = F.gumbel_softmax(adj, tau=0.9, hard=True)

In [72]:
z_1.shape

torch.Size([9900, 2])

In [120]:
z_1[:,0].shape

torch.Size([9900])

In [121]:
edge_index.shape

torch.Size([2, 9900])

In [117]:
z_adj = torch.where(z_1[:,0])

In [118]:
z_adj

(tensor([   2,    3,    4,  ..., 9889, 9890, 9895]),)

In [146]:
b= edge_index[0,:][z_adj]

In [147]:
c = edge_index[1,:][z_adj]

In [157]:
a = torch.stack([b,c])

In [158]:
from torch_geometric.utils import *

In [159]:
is_undirected(a)

True

In [160]:
to_undirected(a)

tensor([[ 0,  0,  0,  ..., 99, 99, 99],
        [ 3,  4,  5,  ..., 88, 89, 94]])